## Procesamiento de Lenguaje Natural

*MINI-TASK \#1* 

# ***Byte-Pair-Encoding del Quijote***

### **Equipo:**

- Giottonini Herrera Enrique Alejandro
- Burruel Durán Luis Andrés
- Jorge Andres Rascon Acuña
- Villalba Miranda Jesús Abraham

**Fuentes**
* [Byte Pair Encoding (Lei Mao) ](https://leimao.github.io/blog/Byte-Pair-Encoding/)
* [Byte-Pair Encoding: Subword-based tokenization algorithm (Chetna Khanna) ](https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0)
---

## **Introducción**

El **byte pair encoding (BPE)** es una técnica sencilla de comprimir datos que consiste en reemplazar la combinación de bytes más frecuente en los datos con un byte que no aparece en ellos.

El objetivo es que al terminar de procesar los datos las palabras más comunes estén representadas en el vocabulario como un solo token, mientras que las palabras raras se dividen en dos o más tokens de subpalabras, lo cual coincide con lo que hace un algoritmo de **subword-based tokenization**.

Los datos que se utilizaran para esta tarea sera _"Don Quijote de la Mancha"_ de _Miguel de Cervantes Saavedra_ en version EBOOK transcrito por _The Project Gutenberg_.

## Aprendiendo de nuestro dataset
Antes de armar el algoritmo de **BPE** primero explicaremos las funciones que necesitamos.

Importamos las librerias que necesitamos

In [1]:
import re, collections

### **Vocab per words**

La función get_vocab se encarga de obtener las palabras contenidas en el vocabulario del corpus y la frecuencia de estas. Para esto, la función recibe el parámetro filename, esto es, el archivo a analizar. 

In [2]:
def get_vocab(filename):
    vocab = collections.defaultdict(int) #Define primero un diccionario vacío llamado "vocab"
    #A continuación, se abre un archivo con la función open, donde la regresa como archivo objeto.
    #Se utiliza como parámetros 'filename', 'r' que indica que se va a leer el archivo (read) y 
    #Por otro lado, utf-8 es un formato de codificación de caracteres Unicode e ISO 10646.
    with open(filename, 'r', encoding='utf-8') as fhand:  
        for line in fhand: #lee cada linea (line)
            words = line.strip().split() #se quitan espacios en blanco y se separan palabras

            #Finalmente, incluye cada palabra en el diccionario y su frecuencia, anexando al final </w>
            for word in words:
                word = re.sub("[.,;\-:!¡¿?]", '', word ) #Quita caracteres especiales
                word = word.lower() #Transforma a minúsculas
                vocab[' '.join(list(word)) + ' </w>'] += 1 
                
    return vocab #Regresa el diccionario "vocab" con el vocabulario

In [3]:
vocabulario = get_vocab('corpus/extract.txt')

In [4]:
print("Vocabulario inicial:")
for word, freq in vocabulario.items():
    print(f"{word}: {freq}")

Vocabulario inicial:
y o </w>: 1
j u a n </w>: 1
g a l l o </w>: 1
d e </w>: 8
a n d r a d a </w>: 1
e s c r i b a n o </w>: 1
c á m a r a </w>: 1
d e l </w>: 4
r e y </w>: 1
n u e s t r o </w>: 1
s e ñ o r </w>: 1
l o s </w>: 2
q u e </w>: 7
r e s i d e n </w>: 1
e n </w>: 4
s u </w>: 1
c o n s e j o </w>: 1
c e r t i f i c o </w>: 1
y </w>: 11
d o y </w>: 1
f e </w>: 1
h a b i e n d o </w>: 1
v i s t o </w>: 1
p o r </w>: 2
s e ñ o r e s </w>: 1
d é l </w>: 1
u n </w>: 1
l i b r o </w>: 4
i n t i t u l a d o </w>: 1
e l </w>: 3
i n g e n i o s o </w>: 1
h i d a l g o </w>: 1
l a </w>: 2
m a n c h a </w>: 1
c o m p u e s t o </w>: 1
m i g u e l </w>: 1
c e r v a n t e s </w>: 1
s a a v e d r a </w>: 1
t a s a r o n </w>: 1
c a d a </w>: 1
p l i e g o </w>: 1
d i c h o </w>: 4
a </w>: 3
t r e s </w>: 2
m a r a v e d í s </w>: 2
m e d i o </w>: 2
c u a l </w>: 1
t i e n e </w>: 1
o c h e n t a </w>: 1
p l i e g o s </w>: 1
a l </w>: 2
p r e c i o </w>: 2
m o n t a </w>: 1
d o c i e n t 

### **Obteniendo los pares**

La función get_stats tiene como parámetro el diccionario vocab generado con la función get_vocab, y devuelve otro diccionario "pairs" con la frecuencia de los pares de tokens más recurrentes.

In [5]:
def get_stats(vocab):
    pairs = collections.defaultdict(int) #Define primero un diccionario vacío llamado "pairs"
    for word, freq in vocab.items():
        symbols = word.split() #Separa la palabra en tokens
        for i in range(len(symbols)-1): #Recorre todos los tokens de la palabra, excluyendo </w>
            #Se suman las veces que se repite una palabra a la frecuencia del par de tokens
            pairs[symbols[i],symbols[i+1]] += freq 
    return pairs #Regresa el diccionario con los pares de tokens mas recurrentes y su frecuencia

In [6]:
pairs = get_stats(vocabulario)

In [7]:
sorted_pairs = sorted(get_stats(vocabulario).items(), key=lambda x:x[1],reverse=True)
for i in range(10):
    print(sorted_pairs[i])

(('o', '</w>'), 29)
(('e', '</w>'), 26)
(('a', '</w>'), 23)
(('d', 'e'), 18)
(('e', 'n'), 17)
(('l', '</w>'), 14)
(('s', '</w>'), 14)
(('y', '</w>'), 13)
(('e', 's'), 12)
(('u', 'e'), 12)


In [8]:
best = max(pairs, key=pairs.get)
print(f"El par de caracteres consecutivos más frecuentes es: {best}")

El par de caracteres consecutivos más frecuentes es: ('o', '</w>')


### **Merge**
La función `merge_vocab` es la encargada de actualizar el vocabulario al mezclar el par de caracteres más frecuente. 

Esta función recibe:
1. **pair**: par más frecuente en el vocabulario
2. **v_in**: el vocabulario.

La función regresa el vocabulario actualizado despues de unir los caracteres y actualizar la frequencia de los tokens.

In [9]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' ' .join(pair)) # ("w1", "w2") -> "w1 w2"
    # Expresión regular utilizada para buscar el par más frecuente de caracteres
    # en el vocabucario"
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

Para probar la función utilizamos el par más frecuente de caracteres de `vocabulario`

In [10]:
new_vocab = merge_vocab(best, vocabulario)
print(f"El par mas frequente: {best}")
print("Nuevo vocabulario:\n")
for word, freq in new_vocab.items():
    print(f"{word}: {freq}")

El par mas frequente: ('o', '</w>')
Nuevo vocabulario:

y o</w>: 1
j u a n </w>: 1
g a l l o</w>: 1
d e </w>: 8
a n d r a d a </w>: 1
e s c r i b a n o</w>: 1
c á m a r a </w>: 1
d e l </w>: 4
r e y </w>: 1
n u e s t r o</w>: 1
s e ñ o r </w>: 1
l o s </w>: 2
q u e </w>: 7
r e s i d e n </w>: 1
e n </w>: 4
s u </w>: 1
c o n s e j o</w>: 1
c e r t i f i c o</w>: 1
y </w>: 11
d o y </w>: 1
f e </w>: 1
h a b i e n d o</w>: 1
v i s t o</w>: 1
p o r </w>: 2
s e ñ o r e s </w>: 1
d é l </w>: 1
u n </w>: 1
l i b r o</w>: 4
i n t i t u l a d o</w>: 1
e l </w>: 3
i n g e n i o s o</w>: 1
h i d a l g o</w>: 1
l a </w>: 2
m a n c h a </w>: 1
c o m p u e s t o</w>: 1
m i g u e l </w>: 1
c e r v a n t e s </w>: 1
s a a v e d r a </w>: 1
t a s a r o n </w>: 1
c a d a </w>: 1
p l i e g o</w>: 1
d i c h o</w>: 4
a </w>: 3
t r e s </w>: 2
m a r a v e d í s </w>: 2
m e d i o</w>: 2
c u a l </w>: 1
t i e n e </w>: 1
o c h e n t a </w>: 1
p l i e g o s </w>: 1
a l </w>: 2
p r e c i o</w>: 2
m o n t a </w>

### **Get tokens**
La función `get_tokens` regresa los los tipos y su frecuencia del vocabulario del corpus.

In [11]:
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

Los tipos de nuestro vocabulario serian

In [12]:
tokens = get_tokens(vocabulario)

Ahora elaboramos una función auxiliar para guardar los tipos y su frecuencia en un archivo de texto

In [13]:
def save_vocab(tokens, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"Tipo, Frecuencia\n")
        for token, freq in tokens.items():
            file.write(f"{token}, {freq}\n")

In [14]:
save_vocab(tokens, 'vocab.txt')

### **Armando el algoritmo**

In [15]:
def byte_pair_encoding(vocab, iter):
    vocab_aux = vocab
    for i in range(iter):
        pairs = get_stats(vocab_aux)
        if not pairs:
            break
        # print(f"Iter {i+1}:")
        best = max(pairs, key=pairs.get)
        vocab_aux = merge_vocab(best, vocab_aux)
        tokens = get_tokens(vocab_aux)
        # print(f"Mejor par {best} \nNúmero de tokens: {len(tokens)}")
        # print('============')
    return vocab_aux

## **Aplicandolo al Quijote**

In [16]:
vocabulario = get_vocab('corpus/quijote.txt')

In [17]:
save_vocab(get_tokens(vocabulario), 'outputs/pretokenization/vocabulario_inicial.txt')

**Número de iteraciones del algoritmo:**

In [18]:
num_merges = 1000

Utilizando el algoritmo

In [33]:
vocabulario = byte_pair_encoding(vocabulario,num_merges)

**Guardamos nuestro vocabulario**

In [34]:
save_vocab(get_tokens(vocabulario), 'outputs/pretokenization/vocab_quijote_1000.txt')

## **Encoding and Decoding**

El ***encoding*** consiste en que dado un vocabulario de tipos $V$ y un corpus/texto $C$ podemos utilizar una función de encoding $E: V'\times C' \to T$ que regresa una lista/secuencia de tokens reconocidos $T$ sobre un corpus $C'$ preprocesado y un vocabulario extendido $V'$ que contiene tokens para casos especiales como cadenas no reconocidas (OOV). De esta manera podemos darle una estructura utilizable al texto. De forma inversa, el ***decoding*** consiste en convertir una lista de tokens en texto.

Definimos una función que pueda leer un diccionario en algun archivo de texto y asegurarse de que los tipos estén ordenados y de filtrar aquellos tokens que tuvieron una frecuencia por debajo de una cota.

In [35]:
def get_vocab_from_path(path: str, lower_bound=0, sortV = lambda list: None, log = lambda *_: None) -> tuple[str]:
    lower_bound = max(lower_bound, 0)
    with open(path) as file:
        vocab = []
        for line in file:
            token, frecuency = line.strip().split(",", 1)
            if isinstance(token, str) and frecuency.strip().isdigit(): # Format from .txt is token, frecuency
                if token[-4:] == '</w>' or int(frecuency) >= lower_bound: # Filter sub-words tokens by lower bound
                    vocab.append(tuple([token, int(frecuency)]))
    sorted_vocab = sortV(vocab)
    log(sorted_vocab, path, lower_bound)
    return tuple(token for token,*_ in sorted_vocab)

Definimos dos funciones de ordenamiento auxiliares:
- Por frecuencia: Nos sirve para saber que tokens aparecierons más veces en el texto.
- Por nombre del token: Es el que usara nuestro ***encoder*** para reconocer el token más largo mientras lee el texto.

Obtenemos la longitud de un token eliminando los últimos caractéres de `</w>`

In [36]:
import os

def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

def sort_by_frecuency(vocabulary: list):
    return sorted(vocabulary, key = lambda item: item[1], reverse=True)

def sort_by_token_name(vocabulary: list):
    return sorted(vocabulary, key = lambda item: (measure_token_length(item[0]), item[1]), reverse=True) # Sort by lenght and then by alphabetic order.
     
def save_vocabulary(vocabulary, path, lower_bound):
    filename = path.rsplit("/")[-1]
    path = os.path.join("vocabularios", f"from_{filename}_with_{lower_bound}_bound")
    with open(path, 'w', encoding='utf-8') as file:
        for token,*_ in vocabulary:
            file.write(token + "\n")

Podemos jugar con el parámetro de `lower_bound` para encontrar aquel que filtre tokens proveniente de sub-palabras que no fueron frecuentes en el algorítmo de byte-pair-encoding.

In [37]:
v = get_vocab_from_path(path="outputs/no_pretokenization/vocab_quijote_8000.txt", lower_bound=300, sortV= sort_by_token_name)

# Sample from vocabulary v from top and bottom
sample_size = 10
bound = int(sample_size / 2)
sample_v = []
sample_v.extend(v[:bound])
sample_v.extend(v[-bound:])
print("---TOP---")
for i, token in enumerate(sample_v):
    if i == bound:
        print("---BOTTOM---")
    print(i+1, token)

---TOP---
1 entretenimiento</w>
2 verdaderamente</w>
3 demasiadamente</w>
4 principalmente</w>
5 entendimiento.</w>
---BOTTOM---
6 s
7 l
8 g
9 a
10 </w>


### Encode de una palabra
La razón por la que el vocabulario fue ordenado por la cantidad de caractéres de los tipos de tokens es porque para realizar el ***encoding*** se va iterar sobre el vocabulario $V$ para encontrar si el token $v_i = c_mc_{m+1}...c_k$ se encuentra en la cadena de caractéres $S = c_1...c_n$ y el ordenamiento asegura que encuentre el tipo de token más largo que capture a la cadena.

In [38]:
import re

def tokenize_word(string, vocabulary, unknown_token='</u>') -> list:
    if string == '':
        return []
    if vocabulary == []:
        return [unknown_token]

    string_tokens = []
    for i in range(len(vocabulary)):
        token = vocabulary[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
  
        if len(matched_positions) == 0:
            continue

        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, vocabulary=vocabulary[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, vocabulary=vocabulary[i+1:], unknown_token=unknown_token)
        break

    if string_tokens == []:
        return [unknown_token]
    return string_tokens

Podemos tokenizar la palabra `facebook`, en la cual el `encoder` es capaz de reconocer los token encerrados en parentesis: `f(a)(c)ebook(<\w>)` y dejando un `</u> ` para toda aquella cadena que sea ***oov*** (out of vocabulary).

In [39]:
word = "facebook</w>"
print( word, "-> [encoder] ->", tokenize_word(word, v, unknown_token='</u>'))


facebook</w> -> [encoder] -> ['</u>', 'a', 'c', '</u>', '</w>']


### Encode de una linea

Definimos una función auxiliar para extender la tokenización de palabras a lineas, y después de lineas a archivos de .txt

In [40]:
def tokenize_line(line: str, vocabulary, unknown_token='</u>') -> list:
    # Data Preprocessing
    # Eliminamos espacios de más
    line = " ".join(line.split())

    # Filtro de caractéres
    line = line.translate({ord(i): None for i in ".;-"})

    # Convertimos/normalizamos el texto en el formato que usa nuestro encoder
    line = line.replace(" ", "</w>")
    
    return tokenize_word(line, vocabulary, unknown_token)

In [41]:
linea = "Esto   es una linea  de ejemplo"
print(tokenize_line(linea, v))

['Esto</w>', 'es</w>', 'una</w>', 'l', 'in', 'ea</w>', 'de</w>', '</u>', 'l', '</u>']


### Encode de un archivo .txt

In [42]:
def encode_text(filename: str, vocabulary, unknown_token='</u>') -> tuple:
    with open(filename, "r", encoding="utf-8") as file:
        return tuple(token for line in file for token in tokenize_line(line, vocabulary))

In [43]:
encoded_text = encode_text("corpus/extract.txt", v)

In [44]:
print(encoded_text)

('</u>', '</w>', 'Juan</w>', '</u>', 'allo</w>', 'de</w>', '</u>', 'd', 'r', 'a', 'd', 'a', '</u>', '</w>', 'escribano</w>', 'de</w>', '</u>', 'ara</w>', 'del</w>', '</u>', 'y</w>', 'nuestro</w>', 's', '</u>', 'r', '</u>', '</w>', 'd', '</u>', 'los</w>', 'que</w>', 're', 's', '</u>', 'den</w>', 'en</w>', 'su</w>', '</u>', 's', '</u>', '</w>', 'c', 'er', 't', '</u>', 'co</w>', 'y</w>', 'doy</w>', 'fe</w>', '</u>', '</w>', 'habiendo</w>', 'visto</w>', '</u>', 'r', 'los</w>', 'señores</w>', 'dél</w>', 'un</w>', 'libro</w>', 'in', 't', '</u>', 't', '</u>', 'lado</w>', 'El</w>', 'ingenioso</w>', 'hidalgo</w>', 'de</w>', 'la</w>', '</u>', 'a', '</u>', 'c', '</u>', 'a', '</u>', 'compuesto</w>', 'por</w>', 'Miguel</w>', 'de</w>', 'Cervantes</w>', '</u>', 'a', 'a', '</u>', 'd', 'r', 'a', '</u>', '</w>', 't', 'a', 's', 'aron</w>', 'cada</w>', 'pliego</w>', 'del</w>', 'di', 'c', '</u>', 'libro</w>', 'a</w>', 'tres</w>', 'maravedís</w>', 'y</w>', 'medio</w>', 'el</w>', 'cual</w>', 'tiene</w>', '</

## Métricas

Proponemos medir el error de un vocabulario $V$ generado por aplicar el algorítmos de byte-pair-encoding. Sea $T$ los tokens generados por un ***encoder*** a un corpus $C$. El error $D$ será: $$D = \dfrac{\text{\#OOV}}{|T|}$$

Donde \#OOV es la cantidad de tokens *out-of-vocabulary*, es decir, no reconocidos en el corpus.

In [45]:
def error_in_vocab(tokens: tuple, unknown_token='</u>', log=lambda *_:None) -> float:
    error = round(tokens.count(unknown_token) / len(tokens), 4)
    log(error, tokens)
    return error

In [46]:
def error_stats(error, token):
    n = len(token)
    print(f"De {n} tokens el {error} son oov.")
    
error = error_in_vocab(encoded_text, log=error_stats)

De 248 tokens el 0.2258 son oov.


### Pruebas

Comparamos el error con la métrica definida arriba para el vocabulario obtenido del Quijote sobre 3 corpus: 1 es otro libro de Cervantes (el autor del Quijote), 1 libro de la epoca y un libro actual.

In [49]:
vocab_no_pretokenization = get_vocab_from_path(path="outputs/no_pretokenization/vocab_quijote_8000.txt", lower_bound=300, sortV= sort_by_token_name)
vocab_pretokenization = get_vocab_from_path(path="outputs/pretokenization/vocab_quijote_8000.txt", lower_bound=300, sortV= sort_by_token_name)

## 1) La Gitanilla
Novela corta de Miguel de Cervantes publicada en 1613.
### Sin pretokenización

In [50]:
# Pasamos el corpus por el encoder
# Computacionalmente caro: 52 min ...
encoded_text = encode_text("corpus/extract_gitanilla.txt", vocab_no_pretokenization)

In [51]:
# Obtenemos el error: 0.2685
error = error_in_vocab(encoded_text, log=error_stats)

De 11585 tokens el 0.2631 son oov.


In [52]:
def save_encoding(tokens: tuple, filename: str):
    with open("encodings/" + filename, 'w', encoding='utf-8') as file:
        for token in tokens:
            file.write(token + ", ")
    print("Saved.")

save_encoding(encoded_text, "gitanilla.1.0")

Saved.


### Con pretokenización

In [53]:
# Pasamos el corpus por el encoder
# Computacionalmente caro: 52 min ...
encoded_text = encode_text("corpus/extract_gitanilla.txt", vocab_pretokenization)

In [54]:
# Obtenemos el error: 0.2685
error = error_in_vocab(encoded_text, log=error_stats)

De 6717 tokens el 0.3189 son oov.


In [55]:
def save_encoding(tokens: tuple, filename: str):
    with open("encodings/" + filename, 'w', encoding='utf-8') as file:
        for token in tokens:
            file.write(token + ", ")
    print("Saved.")

save_encoding(encoded_text, "gitanilla.2.0")

Saved.


## 2) Fuente Ovejuna
Novela de Lope de Vega, rival y amigo de Miguel de Cervantes (lo criticó en la primera parte del Quijote). Publicada en 1619
### Sin pretokenización

In [56]:
# Pasamos el corpus por el encoder
# Computacionalmente caro 26 min
encoded_text = encode_text("corpus/extract_overjuna.txt", vocab_no_pretokenization)

In [57]:
# Obtenemos el error: 0.307
error = error_in_vocab(encoded_text, log=error_stats)

De 5859 tokens el 0.3053 son oov.


In [58]:
save_encoding(encoded_text, "ovejuna.1.0")

Saved.


### Con pretokenización

In [59]:
# Pasamos el corpus por el encoder
# Computacionalmente caro 26 min
encoded_text = encode_text("corpus/extract_overjuna.txt", vocab_pretokenization)

In [60]:
# Obtenemos el error: 0.307
error = error_in_vocab(encoded_text, log=error_stats)

De 2999 tokens el 0.3918 son oov.


In [61]:
save_encoding(encoded_text, "ovejuna.2.0")

Saved.


## 3) El caballero encantado
Novela de Benito Pérez Galdós, escrita en 1909.

In [ ]:
# Pasamos el corpus por el encoder
# Computacionalmente caro 96 min
encoded_text = encode_text("corpus/extract_encantado.txt", vocab_no_pretokenization)

In [ ]:
# Obtenemos el error: 0.2539
error = error_in_vocab(encoded_text, log=error_stats)

In [ ]:
save_encoding(encoded_text, "encantado.1.0")

### Con pretokenization

In [ ]:
# Pasamos el corpus por el encoder
encoded_text = encode_text("corpus/extract_encantado.txt", vocab_pretokenization)

In [ ]:
# Obtenemos el error: 0.2539
error = error_in_vocab(encoded_text, log=error_stats)

In [ ]:
save_encoding(encoded_text, "encantado.2.0")

## **Conclusiones**

Resumiendo los resultados obtenidos:

| **Corpus** | **No pretokenization** | **pretokenization** |
|------------|------------------------|---------------------|
|**La Guitanilla**| 0.2631 | 0.3189 |
| **Fuente ovejuna** | 0.3053  |   |
|**El caballero encantado**|   |   |
